In [103]:
import os

from more_itertools import pairwise, mark_ends
from datetime import datetime
from app.models import BrynzaLine, cast_model
from app.scheduler.brynza.to_boiling_plan import to_boiling_plan as to_boiling_plan_brynza
from app.scheduler.adygea.to_boiling_plan.to_boiling_plan import to_boiling_plan as to_boiling_plan_adygea
from app.scheduler.frontend_utils import draw_excel_frontend
from app.scheduler.time_utils import cast_t
from app.scheduler.wrap_header import wrap_header
from utils_ak.block_tree import BlockMaker, add_push
from utils_ak.iteration import iter_pairs
from utils_ak.numeric import custom_round
from utils_ak.os import open_file_in_os
from utils_ak.pandas import mark_consecutive_groups

In [11]:
# - Read boiling plan

fn = f'/Users/arsenijkadaner/FileApps/coding_projects/umalat/app/data/static/samples/by_department/milk_project/2023-11-19 План по варкам милкпроджект Новый.xlsx'

df1 = boiling_plan_df_brynza = to_boiling_plan_brynza(fn)
df2 = boiling_plan_df_adygea = to_boiling_plan_adygea(fn)

In [13]:
df1.head()

,boiling_id,sku,kg,boiling,batch_id,batch_type
0,1.0,<BrynzaSKU 1>,252.0,<BrynzaBoiling 1>,1.0,brynza
1,1.0,<BrynzaSKU 4>,8.0,<BrynzaBoiling 2>,1.0,brynza


In [14]:
df2.head()

,boiling_id,sku,n_baths,kg,boiling,batch_id,batch_type,absolute_batch_id
0,1,<AdygeaSKU 54>,1,50.0,<AdygeaBoiling 27>,1,adygea,1
1,2,<AdygeaSKU 54>,1,50.0,<AdygeaBoiling 27>,2,adygea,2
2,3,<AdygeaSKU 54>,1,50.0,<AdygeaBoiling 27>,3,adygea,3
3,4,<AdygeaSKU 54>,1,50.0,<AdygeaBoiling 27>,4,adygea,4
4,5,<AdygeaSKU 54>,1,50.0,<AdygeaBoiling 27>,5,adygea,5


In [ ]:
# - Make schedule

# -- Make packings

In [79]:
brynza_line = cast_model(cls=BrynzaLine, obj="Брынза")

start_t = cast_t('11:00')

m = BlockMaker("schedule")

m.row('preparation', size=7)

boiling_technology = df1.iloc[0]['boiling'].boiling_technologies[0]
m.row('packing_brynza', size=round((df1['kg'].sum() / df1['sku'].apply(lambda sku: sku.packing_speed + 175)).sum() * 12))
m.row('small_cleaning', size=5)
m.row('labelling', size=14)

df2['boiling_type'] = df2['boiling'].apply(lambda boiling: str(df2['boiling'].iloc[0].weight_netto) + '-' + str(df2['boiling'].iloc[0].percent))

mark_consecutive_groups(df2, key='boiling_type', groups_key='boiling_type_num')

for is_first, is_last, (i, grp) in mark_ends(list(df2.groupby('boiling_type_num'))):

    boiling = grp.iloc[0]['boiling']
    total_kg = grp['kg'].sum()
    packing_speed = grp['sku'].iloc[0].packing_speed or 0 + 267 # note: packing_speed is the same for all skus in adygea # todo later: check

    # crop to pieces of 200kg

    pieces = [200] * int(total_kg / 200) + [total_kg - 200 * int(total_kg / 200)]

    for _is_first, _is_last, piece in mark_ends(pieces):
        m.row('packing_adygea', size=round(piece / packing_speed * 12))
        if not _is_last:
            m.row('packing_configuration', size=1)

    if not is_last:
        m.row('packing_configuration', size=1)

    m.row('cleaning')

schedule = m.root
schedule

schedule (0, 180] x (0, 0]
  preparation (0, 7] x (0, 0]
  packing_brynza (7, 43] x (0, 0]
  small_cleaning (43, 48] x (0, 0]
  labelling (48, 62] x (0, 0]
  packing_adygea (62, 71] x (0, 0]
  packing_configuration (71, 72] x (0, 0]
  packing_adygea (72, 81] x (0, 0]
  packing_configuration (81, 82] x (0, 0]
  packing_adygea (82, 91] x (0, 0]
  packing_configuration (91, 92] x (0, 0]
  packing_adygea (92, 101] x (0, 0]
  packing_configuration (101, 102] x (0, 0]
  packing_adygea (102, 111] x (0, 0]
  packing_configuration (111, 112] x (0, 0]
  packing_adygea (112, 121] x (0, 0]
  packing_configuration (121, 122] x (0, 0]
  packing_adygea (122, 131] x (0, 0]
  packing_configuration (131, 132] x (0, 0]
  packing_adygea (132, 141] x (0, 0]
  packing_configuration (141, 142] x (0, 0]
  packing_adygea (142, 151] x (0, 0]
  packing_configuration (151, 152] x (0, 0]
  packing_adygea (152, 161] x (0, 0]
  packing_configuration (161, 162] x (0, 0]
  packing_adygea (162, 171] x (0, 0]
  packing_

In [127]:
# - Plot boilings

date = datetime.now()

m = BlockMaker(
    "frontend",
    default_row_width=1,
    default_col_width=1,
    # props
    axis=1,
)
m.row("stub", size=0)  # start with 1

m.block(wrap_header(date=date, start_time='11:00', header="График паковки"))


# make packing line

_m = BlockMaker(
    "packing_line",
    default_row_width=1,
    default_col_width=1,
    # props
    axis=1,
)

for block in schedule.children:
    _block = _m.copy(block, with_props=True)
    _m.block(_block, push_func=add_push, size=(block.size[0], 1))
m.block(_m.root)

frontend = m.root
frontend

frontend (0, 568] x (0, 5]
  stub (0, 0] x (0, 1]
  header (0, 568] x (1, 2]
    block (0, 1] x (1, 2]
    block (1, 2] x (1, 2]
    block (2, 3] x (1, 2]
    block (3, 4] x (1, 2]
    block (4, 5] x (1, 2]
    block (5, 6] x (1, 2]
    block (6, 7] x (1, 2]
    block (7, 8] x (1, 2]
    block (8, 9] x (1, 2]
    block (9, 10] x (1, 2]
    block (10, 11] x (1, 2]
    block (11, 12] x (1, 2]
    block (12, 13] x (1, 2]
    block (13, 14] x (1, 2]
    block (14, 15] x (1, 2]
    block (15, 16] x (1, 2]
    block (16, 17] x (1, 2]
    block (17, 18] x (1, 2]
    block (18, 19] x (1, 2]
    block (19, 20] x (1, 2]
    block (20, 21] x (1, 2]
    block (21, 22] x (1, 2]
    block (22, 23] x (1, 2]
    block (23, 24] x (1, 2]
    block (24, 25] x (1, 2]
    block (25, 26] x (1, 2]
    block (26, 27] x (1, 2]
    block (27, 28] x (1, 2]
    block (28, 29] x (1, 2]
    block (29, 30] x (1, 2]
    block (30, 31] x (1, 2]
    block (31, 32] x (1, 2]
    block (32, 33] x (1, 2]
    block (33, 34]

In [128]:
STYLE = {
    "preparation": {"text": "подготовка, мойка, дезинфекция оборудования. Вызов микробиолога."},
    'packing_brynza': {'text': 'Паковка брынзы'},
    'small_cleaning': {'text': 'мойка'},
    'cleaning': {'text': 'мойка цеха'},
    'packing_configuration': {'color': 'red'},
    'packing_adygea': {'text': 'паковка адыгейского'},
    'labelling': {'text': 'маркировка'}
}


In [129]:
# - Draw frontend

workbook = draw_excel_frontend(
    frontend=frontend,
    style=STYLE,
    open_file=False,
    fn=None,
)


workbook.save("test.xlsx")
open_file_in_os("test.xlsx")


2023-11-23 09:47:08.428 | INFO     | app.scheduler.frontend_utils:draw_schedule:46 - Finished init


In [111]:
frontend

frontend (0, 568] x (0, 3]
  stub (0, 0] x (0, 1]
  header (0, 568] x (1, 2]
    block (0, 1] x (1, 2]
    block (1, 2] x (1, 2]
    block (2, 3] x (1, 2]
    block (3, 4] x (1, 2]
    block (4, 5] x (1, 2]
    block (5, 6] x (1, 2]
    block (6, 7] x (1, 2]
    block (7, 8] x (1, 2]
    block (8, 9] x (1, 2]
    block (9, 10] x (1, 2]
    block (10, 11] x (1, 2]
    block (11, 12] x (1, 2]
    block (12, 13] x (1, 2]
    block (13, 14] x (1, 2]
    block (14, 15] x (1, 2]
    block (15, 16] x (1, 2]
    block (16, 17] x (1, 2]
    block (17, 18] x (1, 2]
    block (18, 19] x (1, 2]
    block (19, 20] x (1, 2]
    block (20, 21] x (1, 2]
    block (21, 22] x (1, 2]
    block (22, 23] x (1, 2]
    block (23, 24] x (1, 2]
    block (24, 25] x (1, 2]
    block (25, 26] x (1, 2]
    block (26, 27] x (1, 2]
    block (27, 28] x (1, 2]
    block (28, 29] x (1, 2]
    block (29, 30] x (1, 2]
    block (30, 31] x (1, 2]
    block (31, 32] x (1, 2]
    block (32, 33] x (1, 2]
    block (33, 34]